Load libraries

In [11]:
require(ggplot2)
#require(GGally)
require(reshape2)
require(lme4)
require(compiler)
require(parallel)
require(boot)
# require(aod)
require(dplyr)

# Load data and set factors

In [38]:
mydata <- read.csv("C:\\Users\\Sarah\\Documents\\Personal Content\\Lab_study_data\\all_massaged_data\\dataframe_all_factors_for_analysis.txt",sep = '\t')
# # SID is the student number
mydata$sid <- factor(mydata$sid)
mydata$sim_index <- factor(mydata$sim_index)
mydata$lab_experience <- factor(mydata$lab_experience)
mydata$similar_sim <- factor(mydata$similar_sim)
mydata$pre_statement <- factor(mydata$pre_statement)
mydata$pre_correct <- factor(mydata$pre_correct)
mydata$similar_sim <- factor(mydata$similar_sim)
mydata$CVS_context <- factor(mydata$CVS_context)

In [39]:
head(mydata)

sid,sim,variable,model,pre_statement,pre_correct,main_statement,main_correct,CVS_context,qual_score,...,use_area,use_separation,activity_order,sim_index,similar_sim,lab_experience,prior_number_virtual_labs,overall.POCC,PC1,PC2
10127163,C,Area,ident,0,NA,0,NA,2,1,...,1,1,LC,2,0,1,1,17,-0.5087537,0.1523806
10127163,C,Area,qual,1,1,1,1,2,1,...,1,1,LC,2,0,1,1,17,-0.5087537,0.1523806
10127163,C,Area,quant,1,1,1,1,2,1,...,1,1,LC,2,0,1,1,17,-0.5087537,0.1523806
10127163,C,Separation,ident,0,NA,0,NA,2,1,...,1,1,LC,2,0,1,1,17,-0.5087537,0.1523806
10127163,C,Separation,qual,1,1,1,1,2,1,...,1,1,LC,2,0,1,1,17,-0.5087537,0.1523806
10127163,C,Separation,quant,1,1,1,1,2,1,...,1,1,LC,2,0,1,1,17,-0.5087537,0.1523806


## Model
By type of worksheet model:

main_correct ~  CVS_context + variable + pre_statement + pre_correct + sim_index + lab_experience + similar_sim + prior_number_virtual_labs

with sid as the student id (8 digit number)

All the variables are binary or categorical except for "CVS_context" and "prior_number_virtual_labs"

https://rstudio-pubs-static.s3.amazonaws.com/33653_57fc7b8e5d484c909b615d8633c01d51.html

Mixed effects with sid as random effects

## All model types together

In [65]:
mixed <- glmer(
    main_statement
    ~  CVS_context*model + variable + sim_index + pre_statement
    + lab_experience + similar_sim + prior_number_virtual_labs + (1 | sid),
           data = mydata, family = binomial, 
           control = glmerControl(optimizer="bobyqa",optCtrl=list(maxfun=200)), nAGQ = 10)

summary(mixed)


Correlation matrix not shown by default, as p = 17 > 12.
Use print(obj, correlation=TRUE)  or
	 vcov(obj)	 if you need it



Generalized linear mixed model fit by maximum likelihood (Adaptive
  Gauss-Hermite Quadrature, nAGQ = 10) [glmerMod]
 Family: binomial  ( logit )
Formula: main_statement ~ CVS_context * model + variable + sim_index +  
    pre_statement + lab_experience + similar_sim + prior_number_virtual_labs +  
    (1 | sid)
   Data: mydata
Control: glmerControl(optimizer = "bobyqa", optCtrl = list(maxfun = 2e+05))

     AIC      BIC   logLik deviance df.resid 
  1455.2   1553.8   -709.6   1419.2     1758 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-3.9956 -0.3828 -0.0985  0.5062 11.7598 

Random effects:
 Groups Name        Variance Std.Dev.
 sid    (Intercept) 0.2265   0.4759  
Number of obs: 1776, groups:  sid, 148

Fixed effects:
                           Estimate Std. Error z value Pr(>|z|)    
(Intercept)               -3.460409   0.385205  -8.983  < 2e-16 ***
CVS_context1              -0.349562   0.536913  -0.651  0.51501    
CVS_context2              -1.767162   0.572141 

## Qual and quant separate

In [44]:
quant_data = mydata[mydata$model == "quant", ]
quant_data = quant_data[complete.cases(quant_data[ , c("main_correct")]), ] #remove data with no main statement for quant model
qual_data = mydata[mydata$model == "qual", ]
qual_data = qual_data[complete.cases(qual_data[ , c("main_correct")]), ] #remove data with no main statement for qual model
ident_data = mydata[mydata$model == "ident", ]
ident_data = ident_data[complete.cases(ident_data[ , c("main_correct")]), ] #remove data with no main statement for ident model

In [66]:
mixed <- glmer(
    main_correct
    ~  CVS_context + variable + sim_index + pre_statement
    + lab_experience + similar_sim + prior_number_virtual_labs + (1 | sid),
           data = qual_data, family = binomial, 
           control = glmerControl(optimizer="bobyqa",optCtrl=list(maxfun=200)), nAGQ = 10)

summary(mixed)

Warning message in commonArgs(par, fn, control, environment()):
"maxfun < 10 * length(par)^2 is not recommended."Warning message in optwrap(optimizer, devfun, start, rho$lower, control = control, :
"convergence code 1 from bobyqa: bobyqa -- maximum number of function evaluations exceeded"Warning message in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv, :
"Model failed to converge with max|grad| = 0.422266 (tol = 0.001, component 1)"

Generalized linear mixed model fit by maximum likelihood (Adaptive
  Gauss-Hermite Quadrature, nAGQ = 10) [glmerMod]
 Family: binomial  ( logit )
Formula: main_correct ~ CVS_context + variable + sim_index + pre_statement +  
    lab_experience + similar_sim + prior_number_virtual_labs +      (1 | sid)
   Data: qual_data
Control: glmerControl(optimizer = "bobyqa", optCtrl = list(maxfun = 200))

     AIC      BIC   logLik deviance df.resid 
   106.9    157.1    -41.5     82.9      471 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-3.2636  0.0021  0.0039  0.0064  0.6723 

Random effects:
 Groups Name        Variance Std.Dev.
 sid    (Intercept) 29.21    5.405   
Number of obs: 483, groups:  sid, 139

Fixed effects:
                          Estimate Std. Error z value Pr(>|z|)
(Intercept)                26.3887   108.9001   0.242    0.809
CVS_context1               -0.7788     1.5767  -0.494    0.621
CVS_context2                1.1976     1.3590   0.881    0.378
variableCo

In [67]:
mixed <- glmer(
    main_correct
    ~  CVS_context + variable + sim_index + pre_statement
    + lab_experience + similar_sim + prior_number_virtual_labs + (1 | sid),
           data = quant_data, family = binomial, 
           control = glmerControl(optimizer="bobyqa",optCtrl=list(maxfun=200)), nAGQ = 10)

summary(mixed)

Warning message in commonArgs(par, fn, control, environment()):
"maxfun < 10 * length(par)^2 is not recommended."Warning message in optwrap(optimizer, devfun, start, rho$lower, control = control, :
"convergence code 1 from bobyqa: bobyqa -- maximum number of function evaluations exceeded"Warning message in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv, :
"Model failed to converge with max|grad| = 0.278256 (tol = 0.001, component 1)"

Generalized linear mixed model fit by maximum likelihood (Adaptive
  Gauss-Hermite Quadrature, nAGQ = 10) [glmerMod]
 Family: binomial  ( logit )
Formula: main_correct ~ CVS_context + variable + sim_index + pre_statement +  
    lab_experience + similar_sim + prior_number_virtual_labs +      (1 | sid)
   Data: quant_data
Control: glmerControl(optimizer = "bobyqa", optCtrl = list(maxfun = 200))

     AIC      BIC   logLik deviance df.resid 
   163.7    207.0    -69.9    139.7      261 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-4.8231  0.0682  0.1045  0.2334  2.5520 

Random effects:
 Groups Name        Variance Std.Dev.
 sid    (Intercept) 2.336    1.528   
Number of obs: 273, groups:  sid, 96

Fixed effects:
                          Estimate Std. Error z value Pr(>|z|)   
(Intercept)                1.78202    1.24456   1.432  0.15219   
CVS_context1              -0.36140    1.23167  -0.293  0.76920   
CVS_context2              -0.80162    0.83435  -0.961  0.33667  

In [69]:
# mixed <- glmer(
#     main_correct
#     ~  CVS_context + variable + sim_index + pre_statement
#     + lab_experience + similar_sim + prior_number_virtual_labs + (1 | sid),
#            data = ident_data, family = binomial, 
#            control = glmerControl(optimizer="bobyqa",optCtrl=list(maxfun=100)), nAGQ = 10)

# summary(mixed)

## Predicting CVS

In [62]:
cvs_data = mydata
cvs_data$CVS_graph = cvs_data$CVS_context
cvs_data$CVS_graph[cvs_data$CVS_graph==1] <- 0
cvs_data$CVS_graph[cvs_data$CVS_graph==2] <- 1

In [73]:
cvs_data$variable <- relevel(cvs_data$variable, "Concentration")

mixed <- glmer(
    CVS_graph
    ~ variable + sim_index + pre_statement*model 
    + lab_experience + similar_sim + prior_number_virtual_labs
    + overall.POCC + PC1 + PC2 + (1 | sid),
           data = cvs_data, family = binomial, 
           control = glmerControl(optimizer="bobyqa",optCtrl=list(maxfun=200)), nAGQ = 10)

summary(mixed)

Warning message in commonArgs(par, fn, control, environment()):
"maxfun < 10 * length(par)^2 is not recommended."Warning message in optwrap(optimizer, devfun, start, rho$lower, control = control, :
"convergence code 1 from bobyqa: bobyqa -- maximum number of function evaluations exceeded"Warning message in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv, :
"Model failed to converge with max|grad| = 0.681577 (tol = 0.001, component 1)"
Correlation matrix not shown by default, as p = 16 > 12.
Use print(obj, correlation=TRUE)  or
	 vcov(obj)	 if you need it



Generalized linear mixed model fit by maximum likelihood (Adaptive
  Gauss-Hermite Quadrature, nAGQ = 10) [glmerMod]
 Family: binomial  ( logit )
Formula: 
CVS_graph ~ variable + sim_index + pre_statement * model + lab_experience +  
    similar_sim + prior_number_virtual_labs + overall.POCC +  
    PC1 + PC2 + (1 | sid)
   Data: cvs_data
Control: glmerControl(optimizer = "bobyqa", optCtrl = list(maxfun = 500))

     AIC      BIC   logLik deviance df.resid 
  1226.3   1319.5   -596.1   1192.3     1759 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-2.8484 -0.1451 -0.0353  0.1484  4.5727 

Random effects:
 Groups Name        Variance Std.Dev.
 sid    (Intercept) 30.25    5.5     
Number of obs: 1776, groups:  sid, 148

Fixed effects:
                          Estimate Std. Error z value Pr(>|z|)    
(Intercept)               -5.76944    3.50842  -1.644  0.10008    
variableArea              -1.14230    0.27538  -4.148 3.35e-05 ***
variableSeparation        -1.26971    0.2

## Modeling transfer

In [80]:
#since we don't include pre data, we filter all rows for different models
transfer_data = mydata[mydata$model == "quant", ]

mixed <- glmer(
    quant_score
    ~ variable + sim_index + CVS_context
    + lab_experience + similar_sim + prior_number_virtual_labs + (1 | sid),
           data = transfer_data, family = binomial, 
           control = glmerControl(optimizer="bobyqa",optCtrl=list(maxfun=200)), nAGQ = 10)

summary(mixed)

Warning message in commonArgs(par, fn, control, environment()):
"maxfun < 10 * length(par)^2 is not recommended."Warning message in optwrap(optimizer, devfun, start, rho$lower, control = control, :
"convergence code 1 from bobyqa: bobyqa -- maximum number of function evaluations exceeded"Warning message in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv, :
"Model failed to converge with max|grad| = 0.0707118 (tol = 0.001, component 1)"

Generalized linear mixed model fit by maximum likelihood (Adaptive
  Gauss-Hermite Quadrature, nAGQ = 10) [glmerMod]
 Family: binomial  ( logit )
Formula: quant_score ~ variable + sim_index + CVS_context + lab_experience +  
    similar_sim + prior_number_virtual_labs + (1 | sid)
   Data: transfer_data
Control: glmerControl(optimizer = "bobyqa", optCtrl = list(maxfun = 200))

     AIC      BIC   logLik deviance df.resid 
   668.9    717.1   -323.5    646.9      581 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-2.5105 -0.4736  0.2939  0.4905  1.6031 

Random effects:
 Groups Name        Variance Std.Dev.
 sid    (Intercept) 3.363    1.834   
Number of obs: 592, groups:  sid, 148

Fixed effects:
                          Estimate Std. Error z value Pr(>|z|)  
(Intercept)                0.70752    0.65021   1.088   0.2765  
variableConcentration     -0.00244    0.33558  -0.007   0.9942  
variableSeparation        -0.70320    0.31096  -2.261   0.0237 *
variableWidth       

In [81]:
#since we don't include pre data, we filter all rows for different models
transfer_data = mydata

mixed <- glmer(
    quant_score
    ~ variable + sim_index + CVS_context + main_statement*model
    + lab_experience + similar_sim + prior_number_virtual_labs + (1 | sid),
           data = transfer_data, family = binomial, 
           control = glmerControl(optimizer="bobyqa",optCtrl=list(maxfun=200)), nAGQ = 10)

summary(mixed)

Warning message in commonArgs(par, fn, control, environment()):
"maxfun < 10 * length(par)^2 is not recommended."Warning message in optwrap(optimizer, devfun, start, rho$lower, control = control, :
"convergence code 1 from bobyqa: bobyqa -- maximum number of function evaluations exceeded"Warning message in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv, :
"Model failed to converge with max|grad| = 0.200572 (tol = 0.001, component 1)"
Correlation matrix not shown by default, as p = 15 > 12.
Use print(obj, correlation=TRUE)  or
	 vcov(obj)	 if you need it



Generalized linear mixed model fit by maximum likelihood (Adaptive
  Gauss-Hermite Quadrature, nAGQ = 10) [glmerMod]
 Family: binomial  ( logit )
Formula: quant_score ~ variable + sim_index + CVS_context + main_statement *  
    model + lab_experience + similar_sim + prior_number_virtual_labs +  
    (1 | sid)
   Data: transfer_data
Control: glmerControl(optimizer = "bobyqa", optCtrl = list(maxfun = 200))

     AIC      BIC   logLik deviance df.resid 
  1511.0   1598.7   -739.5   1479.0     1760 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-3.1572 -0.1831  0.1216  0.4398  2.7837 

Random effects:
 Groups Name        Variance Std.Dev.
 sid    (Intercept) 11.25    3.354   
Number of obs: 1776, groups:  sid, 148

Fixed effects:
                          Estimate Std. Error z value Pr(>|z|)    
(Intercept)                1.04191    0.97963   1.064 0.287522    
variableConcentration     -0.11960    0.22656  -0.528 0.597555    
variableSeparation        -0.91666    0.20638  

## releveling examples

A bunch of relevels

In [ ]:
mydata$variable <- relevel(mydata$variable, "Concentration")

mixed <- glmer(
    CVS_graph_3 
    ~ pre + variable + sim_index +level_experience_sims 
    + experience_undergrad_labs + used_similar_sim + (1 | sid),
           data = mydata, family = binomial, 
           control = glmerControl(optimizer = "bobyqa"), nAGQ = 10)

summary(mixed)

In [ ]:
mydata$variable <- relevel(mydata$variable, "Separation")

mixed <- glmer(
    CVS_graph_3 
    ~ pre + variable + sim_index +level_experience_sims 
    + experience_undergrad_labs + used_similar_sim + (1 | sid),
           data = mydata, family = binomial, 
           control = glmerControl(optimizer = "bobyqa"), nAGQ = 10)

summary(mixed)

In [ ]:
mydata$variable <- relevel(mydata$variable, "Width")

mixed <- glmer(
    CVS_graph_3 
    ~ pre + variable + sim_index +level_experience_sims 
    + experience_undergrad_labs + used_similar_sim + (1 | sid),
           data = mydata, family = binomial, 
           control = glmerControl(optimizer = "bobyqa"), nAGQ = 10)

summary(mixed)